In [12]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from hyperopt import fmin, tpe, hp, STATUS_OK
from hyperopt import SparkTrials
import mlflow

In [13]:
import mlflow
import time
mlflow_uri = "http://mlflow-server:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
mlflow.set_experiment("sample_ds_training")


<Experiment: artifact_location='mlflow-artifacts:/581560746115961737', creation_time=1746775186053, experiment_id='581560746115961737', last_update_time=1746775186053, lifecycle_stage='active', name='sample_ds_training', tags={}>

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sample_ds_training").master("spark://spark-master:7077") \
        .getOrCreate()

In [15]:
# Load the iris dataset from scikit-learn
iris = iris = load_iris()
X = iris.data
y = iris.target

In [16]:
def optimize(C):
    clf = SVC(C = C)

    # Use the cross validation object to compare the model's performance
    accuracy = cross_val_score(clf, X, y).mean()

    return {'loss': -accuracy, 'status': STATUS_OK}

In [17]:
search_space = hp.lognormal('C', 0 , 1.0)
algo = tpe.suggest

In [18]:
from hyperopt import SparkTrials
spark_trials = SparkTrials()

Because the requested parallelism was None or a non-positive value, parallelism will be set to (4), which is Spark's default parallelism (4), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


In [19]:
with mlflow.start_run(run_name='hyperopt') as run:
    argmin = fmin(
    fn= optimize,
    space=search_space,
    algo=algo,
    max_evals = 3,
    trials=spark_trials)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

 67%|██████▋   | 2/3 [00:03<00:01,  1.51s/trial, best loss: -0.9800000000000001]

100%|██████████| 3/3 [00:04<00:00,  1.34s/trial, best loss: -0.9800000000000001]


Total Trials: 3: 3 succeeded, 0 failed, 0 cancelled.


🏃 View run hyperopt at: http://mlflow-server:8888/#/experiments/581560746115961737/runs/691ad8901c074f9f8f696eb8238b65d7
🧪 View experiment at: http://mlflow-server:8888/#/experiments/581560746115961737


In [20]:
# Print the best value found for C
print("Best value found: ", argmin)

Best value found:  {'C': 2.0073893223898645}


In [23]:
spark.stop()